# Table of Contents
* [Exercise (Monte Carlo approximation)](#Exercise-%28Monte-Carlo-approximation%29)
* [Thinking through the problem](#Thinking-through-the-problem)
* [Simple Python](#Simple-Python)
	* [Convergence](#Convergence)
* [Using NumPy](#Using-NumPy)
* [Using Numba](#Using-Numba)
* [Comparing peformance](#Comparing-peformance)


# Exercise (Monte Carlo approximation)

We can use the Python `random` module to perform Monte Carlo simulations.  As a fairly simple application, write a function that returns $\pi$ after approximating it in the manner of the below illustration.

As a bonus, you can choose to examine when convergence of the value occurs after guessing larger numbers of points (perhaps make the function parameterized by such rough convergence and/or by number of guessed points).

<img src="img/Pi_30K.gif"/>

<small>Monte Carlo method applied to approximating the value of π. After placing 30000 random points, the estimate for π is within 0.07% of the actual value. This happens with an approximate probability of 20%.</small>

# Thinking through the problem

Let's think about what the problem requires.  We know from high school geometry that a circle has area $\pi r^2$, and a square has area $a^2$.  For the unit circle, $r$ is 1 and $a$ is 2.  So the circle should enclose $\pi/4$ of the points placed within the unit square.  With that basic math in mind, we can write a solution.

# Simple Python

In [1]:
from random import random
def estimate_pi(N):
    N = int(N)  # Allow passing in floats like 1e6
    inside = 0
    for _ in range(N):
        x, y = random(), random()
        # Since we use the unit circle, we can avoid the sqrt()
        if x**2 + y**2 < 1:
            inside +=1
    # We have looked at one quadrant, so multiply by 4
    return 4*inside/N

In [3]:
estimate_pi(1e8)

3.14151628

## Convergence

Given this function, we can pass in various number of sampled points, and for each determine how close the estimate is to `math.pi`.  Of course, since there is randomness involved, the calculated answer will also vary on each call.

In [4]:
import math
def check_mean_error(N, trials=50):
    answers = []
    for _ in range(trials):
        est = estimate_pi(N)
        answers.append(est)
    mean_err = abs(sum(a-math.pi for a in answers))/N
    return mean_err

In [6]:
check_mean_error(1000)

0.0002523673205103507

# Using NumPy

You may not yet worked with NumPy at this point in the course.  For numeric computation like this, operations using homogeneous arrays of numbers is orders of magnitude faster.  A NumPy solution might look like this:

In [8]:
import numpy as np
xs = np.random.random(100)
ys = np.random.random(100)
inside = np.ones(100)[xs**2 + ys**2 < 1]
inside.size/100

0.81

In [17]:
def estimate_pi_np(N):
    N = int(N)
    xs = np.random.random(N)
    ys = np.random.random(N)
    #np.ones() makes a linear array of N length that is filled with 1s.
    #[xs*xs+ys*ys < 1] calls only the items of the array where this statement is true
    inside = np.ones(N)[xs**2 + ys**2 < 1]
    return 4*inside.size/N    

In [10]:
estimate_pi_np(1e7)

3.14243

# Using Numba

Numba is a Free Software optimizer, built on top of LLVM that can significantly speedup Python code that does numeric computation.  It works as a single decorator on a regular Python function, and can often produce C-like speeds.

In [12]:
from numba import jit

@jit
def estimate_pi_numba(N):
    N = int(N)  # Allow passing in floats like 1e6
    inside = 0
    for _ in range(N):
        x, y = random(), random()
        # Since we use the unit circle, we can avoid the sqrt()
        if x**2 + y**2 < 1:
            inside +=1
    # We have looked at one quadrant, so multiply by 4
    return 4*inside/N

In [13]:
estimate_pi_numba(1e8)

3.14174136

# Comparing peformance

In [14]:
%timeit estimate_pi(1e7)

1 loop, best of 3: 15.1 s per loop


In [15]:
%timeit estimate_pi_np(1e7)

1 loop, best of 3: 1.07 s per loop


In [16]:
%timeit estimate_pi_numba(1e7)

1 loop, best of 3: 553 ms per loop
